# Embeddings

Modele embeddings choisis:
    - Paraphrase-multilingual-MiniLM-L12-v2
    - all-MiniLM-L6-v2


In [6]:
import json
import os

#Analyse sentiment
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import numpy as np

#Embedding
from sentence_transformers import SentenceTransformer

#PostgreSQL
import psycopg2

#MongoDB
from pymongo import MongoClient

#Environnement
from dotenv import load_dotenv

load_dotenv()



True

In [5]:
model_rapide = SentenceTransformer('all-MiniLM-L6-v2')
model_multilingue = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

# Creation de la table embedding et installation de pgvector


### Stockage des embeddings dans la base de données
    Données a stocker:
        - id (Primary key, unique , varchar(30))
        - embedding (vector(384))


### Creation de la table embedding

In [16]:
def base_postgres(requete):
    # Étape 1: Connexion à PostgreSQL
    DB_HOST = os.getenv("DB_HOST")
    DB_PORT = os.getenv("DB_PORT")
    DB_USER = os.getenv("DB_USER")
    DB_PASSWORD = os.getenv("DB_PASSWORD")
    DB_NAME = os.getenv("DB_NAME")
    try:
    # Connexion à la base de données par défaut
        conn = psycopg2.connect(
            host=DB_HOST,
            port=DB_PORT,
            user=DB_USER,
            password=DB_PASSWORD,
            database=DB_NAME
        )
        conn.autocommit = True
        cursor = conn.cursor()
        cursor.execute(requete)
        print("Connexion à PostgreSQL réussie")
    except Exception as e:
        print(f"Erreur lors de la connexion à PostgreSQL: {e}")

In [17]:
requete = "Create table test (id integer)"

base_postgres(requete)


Erreur lors de la connexion à PostgreSQL: relation "test" already exists



### Embedding message 


In [3]:
client = MongoClient('mongodb://localhost:27017/')
result = client['mooc']['documents'].find().limit(10)
for doc in result:
    id = doc.get("id", "")
    message = doc.get("body", "")
    if message == "":
        continue
    print(message)
    result = model_multilingue.encode(message)


Dans le cadre de mes études d'ingénieur j'ai été ammené à aborder la question du management de manière théorique. En poste depuis un peu plus d'un an maintenant, je souhaitais mettre à jour mes connaissances, les confronter aux situations professionnelles que j'ai rencontré. J'aimerais au terme de ce MOOC disposer de compétences manageriales en complément de mon expérience technique.


[-8.70938301e-02  2.15719119e-01  6.74353316e-02  2.65508685e-02
  1.21712079e-02 -1.33489355e-01  1.13881335e-01  1.84810981e-01
 -1.09155267e-01 -1.27696276e-01 -2.77028885e-03  1.01085179e-01
 -3.00158747e-02 -2.23551482e-01  1.57083526e-01  1.24490745e-01
 -7.94572011e-02  5.39578535e-02  6.12663329e-02 -7.69233704e-02
 -1.25018537e-01 -5.48062362e-02  2.40205824e-02 -5.37510552e-02
 -2.02771917e-01  1.56392038e-01  1.09522790e-01 -2.40790427e-01
 -3.77539657e-02 -1.91925719e-01 -3.62726822e-02  1.77031923e-02
  2.35939011e-01 -1.08575851e-01  3.45224664e-02  3.59345704e-01
 -9.14021060e-02  1.465903